### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [2]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    reviews_sorted = reviews.sort_values(by=order_by)
    training_df = reviews_sorted.iloc[:training_size]
    validation_df = reviews_sorted.iloc[training_size:training_size + testing_size]

    return training_df, validation_df

In [4]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [5]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [6]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [7]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

In [8]:
train_data_np

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [9]:
# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.651353
2 		 5.962337
3 		 4.165062
4 		 3.122634
5 		 2.440161
6 		 1.958758
7 		 1.601980
8 		 1.328338
9 		 1.113114
10 		 0.940542
11 		 0.800073
12 		 0.684390
13 		 0.588267
14 		 0.507848
15 		 0.440206
16 		 0.383051
17 		 0.334552
18 		 0.293227
19 		 0.257867
20 		 0.227485
21 		 0.201274
22 		 0.178574
23 		 0.158843
24 		 0.141636
25 		 0.126583
26 		 0.113376
27 		 0.101757
28 		 0.091509
29 		 0.082448
30 		 0.074418
31 		 0.067285
32 		 0.060936
33 		 0.055273
34 		 0.050211
35 		 0.045678
36 		 0.041610
37 		 0.037954
38 		 0.034661
39 		 0.031691
40 		 0.029008
41 		 0.026580
42 		 0.024380
43 		 0.022384
44 		 0.020570
45 		 0.018919
46 		 0.017416
47 		 0.016045
48 		 0.014793
49 		 0.013650
50 		 0.012603
51 		 0.011645
52 		 0.010767
53 		 0.009962
54 		 0.009223
55 		 0.008543
56 		 0.007919
57 		 0.007344
58 		 0.006815
59 		 0.006328
60 		 0.005879
61 		 0.005464
62 		 0.005082
63 		 0.004728
64 		 

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [11]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Use the training data to create a series of users and movies that matches the ordering in training data
    users_list = np.array(train_data_df.index)
    movies_list = np.array(train_data_df.columns)

    # User row and Movie Column
    row_idx = np.where(users_list == user_id)[0][0]
    column_idx = np.where(movies_list == movie_id)[0][0]
    print(row_idx, column_idx)
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(user_matrix[row_idx, :], movie_matrix[:, column_idx])
   
    return pred

In [12]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

0 0


6.111887409464288

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [13]:
str(movies[movies['movie_id'] == 2844]['movie'])

"15    Fantômas - À l'ombre de la guillotine (1913)\nName: movie, dtype: object"

In [14]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    movie_name = str(movies[movies['movie_id'] == movie_id]['movie'])[6:].replace('\nName: movie, dtype: object', '')
    print('User: %s, Movie: %s, Predicted Rating: %.2f' % (user_id, movie_name, prediction))


In [15]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

User: 8, Movie: Fantômas - À l'ombre de la guillotine (1913), Predicted Rating: 6.11


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [16]:
val_df.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
650588,49056,1598822,8,1363308721,2013-03-15 00:52:01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650569,49056,289879,9,1363308742,2013-03-15 00:52:22,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650585,49056,1563738,9,1363308780,2013-03-15 00:53:00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650583,49056,1458175,4,1363308799,2013-03-15 00:53:19,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
378686,28599,103639,8,1363309112,2013-03-15 00:58:32,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    user_id, movie_id, rating = val_df['user_id'].tolist(), val_df['movie_id'].tolist(), val_df['rating'].tolist()
    
    for i in range(num_preds):
            pred = predict_rating(user_mat, movie_mat, user_id[i], movie_id[i])
            print('User: %d, Movie: %d, Predicted Rating: %.2f, Actual Rating: %.f' % (user_id[i],
                                                                                      movie_id[i],
                                                                                      pred, rating[i]))
        
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

2966 2191
User: 49056, Movie: 1598822, Predicted Rating: 7.51, Actual Rating: 8
2966 973
User: 49056, Movie: 289879, Predicted Rating: 7.95, Actual Rating: 9
2966 2142
User: 49056, Movie: 1563738, Predicted Rating: 8.90, Actual Rating: 9
2966 2063
User: 49056, Movie: 1458175, Predicted Rating: 7.20, Actual Rating: 4
1726 450
User: 28599, Movie: 103639, Predicted Rating: 7.42, Actual Rating: 8
3065 2141
User: 50593, Movie: 1560985, Predicted Rating: 3.97, Actual Rating: 4


In [18]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 20)        

2966 2191
User: 49056, Movie: 1598822, Predicted Rating: 7.51, Actual Rating: 8
2966 973
User: 49056, Movie: 289879, Predicted Rating: 7.95, Actual Rating: 9
2966 2142
User: 49056, Movie: 1563738, Predicted Rating: 8.90, Actual Rating: 9
2966 2063
User: 49056, Movie: 1458175, Predicted Rating: 7.20, Actual Rating: 4
1726 450
User: 28599, Movie: 103639, Predicted Rating: 7.42, Actual Rating: 8
3065 2141
User: 50593, Movie: 1560985, Predicted Rating: 3.97, Actual Rating: 4


IndexError: index 0 is out of bounds for axis 0 with size 0

** A statement about why you think what happened happened.**

either the user/movie is not in the training set -> cannot make predictions